# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [117]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
city_csv = "../../WeatherPy/starter_code/output_data/cities.csv"
city_df = pd.read_csv(city_csv)

city_df = city_df[["City","Cloudiness","Country","Date", "Humidity",
       "Latitude", "Longitude", "Max Temp", "Wind Speed"]]
city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Tuktoyaktuk,75,CA,1612571511,75,69.4541,-133.0374,-31.00,10.36
1,Ushuaia,75,AR,1612571421,76,-54.8000,-68.3000,46.40,26.46
2,Wanning,50,CN,1612571597,83,18.8003,110.3967,70.16,5.19
3,Rovaniemi,90,FI,1612571597,92,66.5000,25.7167,15.80,9.22
4,Hobart,75,AU,1612571484,60,-42.8794,147.3294,75.00,13.80
...,...,...,...,...,...,...,...,...,...
588,Norton,73,ZW,1612571699,87,-17.8833,30.7000,64.99,1.99
589,San Quintín,0,MX,1612571699,75,30.4833,-115.9500,59.50,13.09
590,Riacho de Santana,100,BR,1612571700,83,-13.6092,-42.9389,72.82,4.16
591,Catazaja,0,MX,1612571700,80,17.7250,-92.0167,72.10,4.09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [60]:
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '100%',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

humidity_locations = city_df[["Latitude", "Longitude"]]
humidity_weight = city_df["Humidity"]

humidity_fig = gmaps.figure(layout=figure_layout)
humidity_heat_layer = gmaps.heatmap_layer(humidity_locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
humidity_fig.add_layer(humidity_heat_layer)
humidity_fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [149]:
max_temp_df = city_df[city_df["Max Temp"] >= 78]
cloudiness_df = max_temp_df[max_temp_df["Cloudiness"] <= 50]
ideal_weather_df = cloudiness_df[cloudiness_df["Humidity"] <= 50]
ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
59,Teuchitlán,0,MX,1612571606,21,20.6833,-103.8667,78.01,7.00
170,Puruándiro,20,MX,1612571622,27,20.0833,-101.5000,78.80,17.27
202,San Rafael,0,AR,1612571628,27,-34.6177,-68.3301,79.41,6.58
304,Acajutla,0,SV,1612571652,47,13.5928,-89.8275,82.40,2.30
321,Cabo San Lucas,20,MX,1612571656,36,22.8909,-109.9124,80.60,3.44
350,Mandiana,45,GN,1612571662,19,10.6333,-8.6833,78.42,3.29
531,Caucel,1,MX,1612571686,45,21.0167,-89.7167,86.00,12.66
541,Agua Verde,1,MX,1612571688,50,22.9000,-106.0500,78.80,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [161]:
hotel_name = []

teu_latitude = 20.6833
teu_longitude = -103.8667

puru_latitude = 20.0833
puru_longitude = -101.5000

sanraf_latitude = -34.6177
sanraf_longitude = -68.3301

aca_latitude = 13.5928
aca_longitude = -89.8275

cabo_latitude = 22.8909
cabo_longitude = -109.9124

mand_latitude = 10.6333
mand_longitude = -8.6833

cau_latitude = 21.0167
cau_longitude = -89.7167

agua_latitude = 22.9000
agua_longitude = -106.0500

radius = 5000
keyword = "hotel"
key = g_key

teu_parameters = {"location": f"{teu_latitude},{teu_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

puru_parameters = {"location": f"{puru_latitude},{puru_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

sanraf_parameters = {"location": f"{sanraf_latitude},{sanraf_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

aca_parameters = {"location": f"{aca_latitude},{aca_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

cabo_parameters = {"location": f"{cabo_latitude},{cabo_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

mand_parameters = {"location": f"{mand_latitude},{mand_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

cau_parameters = {"location": f"{cau_latitude},{cau_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

agua_parameters = {"location": f"{agua_latitude},{agua_longitude}",
             "radius": radius,
             "keyword": keyword,
             "key": key}

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"

teu_response = requests.get(url, params=teu_parameters).json()
hotel_name.append(teu_response["results"][0]["name"])

puru_response = requests.get(url, params=puru_parameters).json()
hotel_name.append(puru_response["results"][0]["name"])

sanraf_response = requests.get(url, params=sanraf_parameters).json()
hotel_name.append(sanraf_response["results"][0]["name"])

aca_response = requests.get(url, params=aca_parameters).json()
hotel_name.append(aca_response["results"][0]["name"])

cabo_response = requests.get(url, params=cabo_parameters).json()
hotel_name.append(cabo_response["results"][0]["name"])

mand_response = requests.get(url, params=mand_parameters).json()
hotel_name.append(mand_response["results"][0]["name"])

try:
    cau_response = requests.get(url, params=cau_parameters).json()
    hotel_name.append(cau_response["results"][0]["name"])
except:
    hotel_name.append("No hotel found")

try:
    agua_response = requests.get(url, params=agua_parameters).json()
    hotel_name.append(agua_response["results"][0]["name"])
except:
    hotel_name.append("No hotel found")
    
print(hotel_name)
#print(json.dumps(response, indent=4, sort_keys=True))

['Hacienda El Carmen Hotel & Spa', 'HOTEL LOMA LINDA', 'Tower Inn Hotel & Suites Casino Spa Convention Center', 'Villavela Hotel Boutique', 'Waldorf Astoria Los Cabos Pedregal', "Centre d'accueil de Mandiana", 'No hotel found', 'No hotel found']


In [168]:
#Drop cities with no hotels
hotel_df = ideal_weather_df.assign(Hotel = hotel_name)
hotel_df = hotel_df[hotel_df["Hotel"] != "No hotel found"]
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel
59,Teuchitlán,0,MX,1612571606,21,20.6833,-103.8667,78.01,7.00,Hacienda El Carmen Hotel & Spa
170,Puruándiro,20,MX,1612571622,27,20.0833,-101.5000,78.80,17.27,HOTEL LOMA LINDA
202,San Rafael,0,AR,1612571628,27,-34.6177,-68.3301,79.41,6.58,Tower Inn Hotel & Suites Casino Spa Convention...
304,Acajutla,0,SV,1612571652,47,13.5928,-89.8275,82.40,2.30,Villavela Hotel Boutique
321,Cabo San Lucas,20,MX,1612571656,36,22.8909,-109.9124,80.60,3.44,Waldorf Astoria Los Cabos Pedregal
350,Mandiana,45,GN,1612571662,19,10.6333,-8.6833,78.42,3.29,Centre d'accueil de Mandiana


In [134]:
# hotel_name = []
# latitude = ideal_weather_df["Latitude"]
# longitude = ideal_weather_df["Longitude"]
# cities = ideal_weather_df["City"]
# radius = 5000
# keyword = "hotel"
# key = g_key

# parameters = {"location": f"{latitude},{longitude}",
#              "radius": radius,
#              "keyword": keyword,
#              "key": key}


# for city in cities:
#     try:
#         url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"
#         response = requests.get(url, params=parameters).json()
#         hotel_name.append(response["results"][0]["name"])
#         print(hotel_name)
#     except:
#         print(f"No hotel found for {city}")
#     print(response)
    
# hotel_name.append(response.json())
# print(hotel_name)

Hotel not found for Teuchitlán
Hotel not found for Puruándiro
Hotel not found for San Rafael
Hotel not found for Acajutla
Hotel not found for Cabo San Lucas
Hotel not found for Mandiana
Hotel not found for Caucel
Hotel not found for Agua Verde


In [169]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [173]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
humidity_fig.add_layer(markers)

# Display figure
humidity_fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…